In [115]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [116]:
!pip install -U flask-cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [117]:
from flask import Flask,request,render_template,url_for,jsonify
from flask_cors import CORS, cross_origin
import site
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score
from gensim.models.keyedvectors import KeyedVectors
from keras import backend as K

In [118]:
# Generating word tokens after removing characters other than alphabets, converting them to lower case and
# removing stopwords from the text'''

def word_tokens(essay_text):
    essay_text = re.sub("[^a-zA-Z]", " ", essay_text)
    words = essay_text.lower().split()
    stop_words = set(stopwords.words("english"))
    words = [w for w in words if not w in stop_words]
    return (words)

In [119]:
# Generating sentence tokens from the essay and finally the word tokens

def sentence_tokens(essay_text):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sent_tokens = tokenizer.tokenize(essay_text.strip())
    sentences = []
    for sent_token in sent_tokens:
        if len(sent_token) > 0:
            sentences.append(word_tokens(sent_token))
    return sentences

In [120]:
# Generating a vector of features

def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

In [121]:
# Generating word vectors to be used in word2vec model

def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay_text in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay_text, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [122]:
def convertToVec(text):
    content=text
    if len(content) > 20:
        num_features = 300
        model = KeyedVectors.load_word2vec_format("/content/word2vecmodel_lstm.bin", binary=True)
        clean_test_essays = []
        clean_test_essays.append(word_tokens(content))
        testDataVecs = getAvgFeatureVecs(clean_test_essays, model, num_features )
        testDataVecs = np.array(testDataVecs)
        testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
        lstm_model = load_model("/content/LSTM.h5")
        preds = lstm_model.predict(testDataVecs)
        print(preds)
        return str(round(preds[0][0]))


In [129]:

def create_task():
    K.clear_session()
    test = input("Enter eassy : ") 
    #test="The Newspaper is one of the oldest means of communication which provides information from all around the world."
    score = convertToVec(test)
    K.clear_session()
    return score


In [130]:
sc=create_task()
print("Score : ",sc)

Enter eassy : I think the computer dire a postive effect on people all over the world. You can do lots of things on the computer like an acidemic work, talk on the enernet, or even find out facts' about fam'us people you love, people computer. I think computer can benfit socity becuse computers can help you with your acidimic work, and can also talk to you friends online. One way i can benfit from using the computer by, typing my school work. But when in not doing my school work talking to my friends online. But the best thing of all is finding out information about famuse people like @PERSON1. I rember when is was doing a project on the computer and got a @NUM1 on it becuse i used the computer. That should prove that computers are very help full. Have you ever wondered how a computer can help you in your ever day life. If you said no i can tell you how. You can learn about diffrent places and food. Lets say i was going to make pizza and i did not know how i would have to look on the c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[[7.882696]]
Score :  8
